We first call the packages that is going to be used


In [ ]:
import pandas as pd
from datetime import date, timedelta, datetime
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

Here we define a funcion that will read the dates and return the data from the MTA site and save it as a data frame. The dates formated in order to be used in the next function.

In [ ]:
def download_MTA_data(start_ = date(2014, 10, 25), end_= datetime.date(datetime.now())): 
    df_url = pd.DataFrame()
    for i in pd.date_range(start=start_, end=end_, freq='7D'):
        ii = str(i)
        formated_date = ii[2:4] + ii[5:7] + ii[8:10]
        df_url = df_url.append(pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_"+formated_date+".txt"))
    return df_url.reset_index()

Note that the starting date must be in the list shown in: http://web.mta.info/developers/turnstile.html.



The starting date should also be on October 25, 2014, or after that but not before. The reason is that the data structure differs in files uploaded before that which will correspond to errors and nullities in the data.

In [ ]:
start_date = date(2016, 9, 3)
end_date = date(2016, 12, 17)
df = download_MTA_data(start_date, end_date)

In [ ]:
# To create a database, we fisrt downloaded the data as a csv file

In [ ]:
# file_name = 'C:/Users/Windows10/SDAIA_Bootcamp/NBM_EDA_Gamma/EDA_MVP_Moh_Os.csv'
# df.to_csv(file_name, index=False)

In [ ]:
# Then we created a local database using SQLite. Now, we import the data and use it.

In [ ]:
# engine = create_engine("sqlite:///eda_1.db")

In [ ]:
# df = pd.read_sql('SELECT * FROM df;', engine)

In [ ]:
df.head()

In [ ]:
# Check the type of the date and time columns
print(df[['DATE', 'TIME']].dtypes)

In [ ]:
# We now want to fisrt create a 'date_time' column and convert the 'date' 
# column from object to  date type 
df['DATE_TIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
df['DATE'] = pd.to_datetime(df['DATE'])

In [ ]:
print(df[['DATE_TIME']].dtypes)
print(df[['DATE']].dtypes)

Each compination of C/A, UNIT, SCP, and STATION represents a unique turnstile. Where:
<br>C/A: Control Area.
<br>UNIT: Remote Unit for a station.
<br>SCP: Subunit Channel Position represents an specific address for a device.
<br>STATION: Represents the station name the device is located at.


In [ ]:
# Let us first clear the columns names so that we can use it.
df.columns

In [ ]:
# Here we clean columns names and remove all unnecessary spaces.
df.columns = [column.strip() for column in df.columns]
df.columns

Date is recorded every 4 hours, and the ENTRIES and EXITS columns are cumulative.

In [ ]:
# To exploring the data, we will group by C/A, UNIT, SCP, STATION and DATE and get 
# entries and exits. This data represends the commulative ENTRIES and EXITS for each turnstile every
# four hours.
turnstile_df = df.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'DATE_TIME'],as_index=False)[['ENTRIES', 'EXITS']].min() 
turnstile_df.head()

We want to find the difference between the ENTRIES_PFH and the EXITS_PFH. This way, we can know the number of people inside a specific station in 4 hours duration.

But first, we need to clean the new columns and make sure data is correct.

In [ ]:
# we may face some cases where the ENTRIES or EXITS are negative, for these situations we will 
# deal with it using the following:

# First, since we know each compination of C/A, UNIT, SCP, and STATION represents a unique turnstile.
# We will force the value of entries equals to zero when it is subtracted from different turnstiles.
turnstile_df['TURNSTILE_ID'] = turnstile_df["C/A"] + turnstile_df["UNIT"] + turnstile_df["SCP"] + turnstile_df["STATION"]

turnstile_df['ENTRIES_PFH']=np.where(turnstile_df['TURNSTILE_ID'] == turnstile_df["TURNSTILE_ID"].shift(-1), 
                                     turnstile_df['ENTRIES'].diff().shift(-1) ,
                                     0)
turnstile_df['EXITS_PFH']=np.where(turnstile_df['TURNSTILE_ID'] == turnstile_df["TURNSTILE_ID"].shift(-1), 
                                   turnstile_df['EXITS'].diff().shift(-1),
                                   0)

In [ ]:
# Check if there are negative values
turnstile_df[(turnstile_df.ENTRIES_PFH < 0) | (turnstile_df.EXITS_PFH < 0)]

In [ ]:
# Now we deal with negative numbers
turnstile_df.loc[turnstile_df.ENTRIES_PFH < 0,'ENTRIES_PFH'] = abs(turnstile_df[turnstile_df.ENTRIES_PFH < 0].ENTRIES_PFH)          
turnstile_df.loc[turnstile_df.EXITS_PFH < 0,'EXITS_PFH'] = abs(turnstile_df[turnstile_df.EXITS_PFH < 0].EXITS_PFH)          

In [ ]:
# Check if there are negative values
turnstile_df[(turnstile_df.ENTRIES_PFH < 0) | (turnstile_df.EXITS_PFH < 0)]

#### Check for outliers 
we will create a threshold of (14,400) that means it is impossible for a single turnstile to have more than one person per seconds.



In [ ]:
threshold = 14400
turnstile_df = turnstile_df[(turnstile_df.ENTRIES_PFH < threshold)|(turnstile_df.EXITS_PFH < threshold)]

In [ ]:
# Create a new column representing the number of people inside counted by a turnstile
turnstile_df['PEOPLE_INSIDE'] = turnstile_df['ENTRIES_PFH'] - turnstile_df['EXITS_PFH']

let us now group the turnstile dataframe by stations so that we see the daily entries of that station.


In [ ]:
station_df = turnstile_df.groupby(by=['STATION', 'DATE'],as_index=False)[['ENTRIES_PFH', 'EXITS_PFH', 'PEOPLE_INSIDE']].sum()

We will now take the absolute value of the data 'PEOPLE_INSIDE' column to know how many people were there in the station in the four hours period. The reason why we considered negative values in the 'PEOPLE_INSIDE' column is that we are looking for people inside the station, either arriving or going to get into the subway.

In [ ]:
station_df.loc[station_df.PEOPLE_INSIDE < 0,'PEOPLE_INSIDE'] = abs(station_df[station_df.PEOPLE_INSIDE < 0].PEOPLE_INSIDE) 

In [ ]:
station_df

In [ ]:
# lets see the boxplot of the'PEOPLE_INSIDE' columns for a station '1 AV'
f, ax = plt.subplots(figsize=(12, 9))
ax.tick_params(axis='x', rotation=90)

sns.boxplot(station_df[(station_df['PEOPLE_INSIDE']<2600)&(station_df['STATION']=='1 AV')]['STATION'], station_df[(station_df['PEOPLE_INSIDE']<2600)&(station_df['STATION']=='1 AV')]['PEOPLE_INSIDE'])


In [ ]:
# let us select a station '1 AV' and see the plot of the PEOPLE_INSIDE column

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(station_df[station_df['STATION'] == '1 AV']['DATE'], station_df[station_df['STATION'] == '1 AV']['PEOPLE_INSIDE'])
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=15)
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Entries')
plt.title('Station 1AV Daily Entries', fontsize=25)
plt.show()

In [ ]:
# Let's see it more closely, for few weeks

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(station_df[(station_df['STATION'] == '1 AV')&
                    (station_df['DATE'] > '2016-09-01')&
                    (station_df['DATE'] < '2016-09-23')]['DATE'], 
         station_df[(station_df['STATION'] == '1 AV')&
                    (station_df['DATE'] > '2016-09-01')&
                    (station_df['DATE'] < '2016-09-23')]['PEOPLE_INSIDE'])
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=15)
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Entries')
plt.title('Station 1AV Daily Entries', fontsize=25)
plt.show()

In [ ]:
# Let's see it more closely, for few weeks and for all stations

In [ ]:
station_df['DAYS'] = pd.to_datetime(station_df['DATE']).dt.dayofweek
station_df['WEEK'] =  pd.to_datetime(station_df['DATE']).dt.week

In [ ]:
days_to_plot = station_df.groupby(["WEEK","DAYS"])[['PEOPLE_INSIDE']].sum()
days_to_plot.index[[0,1,2][2]][1]

In [ ]:
plt.figure(figsize=(25,5));
plt.xlabel("DAYS")
plt.ylabel("ENTRIES PER DAY")
#ignore week 34
start_indx = days_to_plot.index[0]

for i in range(15):
    plt.plot(days_to_plot[days_to_plot['PEOPLE_INSIDE'] < 0.2e7].loc[35+i]);

We can see from the plot that the people inside column reaches is maximum (in most cases) in weekdays. So, we will focus on these days when giving the recomendations.